# Curation Estaciones Meteorologicas

Estudiar La cantidad de datos validos disponibles para cada variable en cada estacion y obtener el numero de estaciones con los datos minimos necesarios para el estudio

In [1]:
setwd("~/Repositories/AirQualityCOVID")

In [2]:
source("src/general.R")

## Funciones para filtrar datos

In [3]:
suppressMessages(library(lubridate))
suppressMessages(library(tidyverse))
suppressMessages(library(plyr))

In [17]:
get.info <- function(st) {
    
    dataframe <- read.csv(paste("data/Curation/AEMET/Values/",
                                st,
                                ".csv", sep=""))
    
    dataframe$fecha <- ymd(dataframe$fecha)
    period <- as.numeric(ymd("2020-12-31")-ymd("2013-01-01")) +2
    print(period)
    
    cbind(data.frame(site=st,
                     start_dt=min(dataframe$fecha),
                     end_dt=max(dataframe$fecha),
                     miss_dy=period - nrow(dataframe)
                    ),
          rbind(apply(dataframe, 2, 
                      function(df, period) {
                          (sum(!is.na(df)) / period)
                      }, period))
         )
}

In [18]:
sitesAQ <- read.csv("data/Curation/sitesAQ.csv",
                    stringsAsFactor=TRUE)

all.data <- do.call(rbind.fill, lapply(levels(sitesAQ$site), get.info))

[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923
[1] 2923


In [21]:
(all.data$miss_dy)

[1]    0    0    0    0  106    0    0    0   35    0    0    0    0 1704    0
[16]    0    0    0    0    0    0  283    0    0    9  283    0    0   10    0
[31]    0    0    0    0  106  106    0    0    0    0  339    0    0    8    0
[46]    0    0    0    0    0   63    0   35   35    0    0  106  106    0    0
[61]    1    0    0    8    1    0    0

# Guardar Datos

```R
write.csv(all.data,
          "data/Curation/AEMET/info_sites.csv",
          row.names=F)
```

In [13]:
info <- data.frame()
percent <- c(0.5, 0.7, 0.8, 0.9)

for (pr in percent) {
    info <- rbind(info,
                  rbind(apply(all.data, 2, 
                              function(df){
                                  sum(df>= pr, na.rm=TRUE)
                              }))[10:ncol(all.data)]
                 )
}

names(info) <- names(all.data)[10:ncol(all.data)]
row.names(info) <- percent

In [14]:
info

,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin,siteAQ
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0.5,66,66,66,61,66,61,53,54,53,59,50,55,55,55,55,66
0.7,66,66,66,61,66,61,53,53,53,59,49,55,55,55,55,66
0.8,65,66,65,61,65,61,53,53,53,59,46,55,55,55,55,66
0.9,58,63,58,60,58,60,50,50,50,58,46,52,52,52,52,65
